In [1]:
import os
from llm_workflow.hugging_face import HuggingFaceEndpointChat, get_tokenizer, num_tokens
from IPython.display import Markdown, display
from dotenv import load_dotenv
load_dotenv()

tokenizer = get_tokenizer('meta-llama/Llama-2-7b-chat-hf')
def cal_num_tokens(text: str) -> int:
    """Use the tokenizer to count the number of tokens in the text."""
    return num_tokens(text, tokenizer)

model = HuggingFaceEndpointChat(
    endpoint_url=os.getenv('HUGGING_FACE_ENDPOINT_LLAMA2_7B'),
    token_calculator=cal_num_tokens,
    streaming_callback=lambda x: print(x.response, end='||', flush=True),
)

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model("Hi, my name is Shane. What's your name?")

Hello Shane! *smiling* My name|| is LLaMA, I'm a helpful|| AI assistant developed by Meta AI. How|| can I assist you today? Is there something you||'d like to chat about or a task you||'d like me to help you with?||

"Hello Shane! *smiling* My name is LLaMA, I'm a helpful AI assistant developed by Meta AI. How can I assist you today? Is there something you'd like to chat about or a task you'd like me to help you with?"

In [3]:
model.total_tokens

108

In [4]:
model("What's my name?")

Ah, great! *smiling* Your name|| is Shane. *nodding* Is|| there anything else you'd like to know or|| discuss? I'm here to help with any|| questions or tasks you may have.||

"Ah, great! *smiling* Your name is Shane. *nodding* Is there anything else you'd like to know or discuss? I'm here to help with any questions or tasks you may have."

---

In [5]:
model = HuggingFaceEndpointChat(
    endpoint_url=os.getenv('HUGGING_FACE_ENDPOINT_LLAMA2_7B'),
    streaming_callback=lambda x: print(x.response, end='||', flush=True)
)

In [6]:
response = model("Write a python function to sum two numbers.")
display(Markdown(response))

```
def sum(a, b):
||    return a + b
```
You can|| call the function by passing in the two numbers you|| want to sum, like this:
```
||result = sum(3, 5)
||print(result) # Output: 8
||```
I hope this helps! Let me know|| if you have any other questions.||

```
def sum(a, b):
    return a + b
```
You can call the function by passing in the two numbers you want to sum, like this:
```
result = sum(3, 5)
print(result) # Output: 8
```
I hope this helps! Let me know if you have any other questions.

In [7]:
response = model("Update the function to use a doc-string and type hints.")
display(Markdown(response))

```
def sum(a: int, b||: int) -> int:
    """
||    Returns the sum of two numbers.

||    :param a: The first number to sum||.
    :param b: The second number|| to sum.
    :return: The sum|| of a and b.
    """
   || return a + b
```
I added a|| doc-string to the function, which is a|| brief description of what the function does and what it|| takes as input. I also added type hints for|| the function parameters and return value, which can help|| make the code more readable and easier to understand.||

You can call the function in the same|| way as before:
```
result = sum||(3, 5)
print(result||) # Output: 8
```
I|| hope this helps! Let me know if you have|| any other questions.||

```
def sum(a: int, b: int) -> int:
    """
    Returns the sum of two numbers.

    :param a: The first number to sum.
    :param b: The second number to sum.
    :return: The sum of a and b.
    """
    return a + b
```
I added a doc-string to the function, which is a brief description of what the function does and what it takes as input. I also added type hints for the function parameters and return value, which can help make the code more readable and easier to understand.

You can call the function in the same way as before:
```
result = sum(3, 5)
print(result) # Output: 8
```
I hope this helps! Let me know if you have any other questions.

---